# Celery worker setup

1) add celery.py to the main app
2) add this to setting.py  CELERY_BROKER_URL = os.getenv("CELERY_BROKER_URL", "redis://redis:6379/0")
3) add tasks.py to your app and use @shared_task decorator to define it as a task   

# Independed Celery worker

1) make a new directory next to yor project dir 
2) add a celerytask.py and celeryconfig.py to it
3) add Dockerfile and requirement.txt if you want to build it seperately
4) in celerytask.py make a new celery instance and choose celeryconfig as a setting file 
    app = Celery('mytask')
    app.config_from_object('celeryconfig')
    @app.task
    def your_task():
        return

5) add congifs to the celeryconfig.py
    broker_url = 'redis://redis:6379/0'
    result_backend = 'redis://redis:6379/0'

6) add a service using this directory to the docker compose file

## Celery settings

In [ ]:
'''-------------- celery@44b015f2d5cf v5.5.1 (immunity)
 --- ***** ----- 
 -- ******* ---- Linux-6.10.14-linuxkit-x86_64-with-glibc2.36 2025-04-12 06:45:38
 - *** --- * --- 
 - ** ---------- [config]
 - ** ---------- .> app:         mytask:0x7f0ddfae1c00
 - ** ---------- .> transport:   redis://redis:6379/0   -----------------------------------> these two lines must be redis
 - ** ---------- .> results:     redis://redis:6379/0   -----------------------------------> these two lines must be redis
 - *** --- * --- .> concurrency: 16 (prefork)
 -- ******* ---- .> task events: OFF (enable -E to monitor tasks in this worker)
 --- ***** ----- 
  -------------- [queues]
                 .> celery           exchange=celery(direct) key=celery
                 
 
 [tasks]
   . celerytask.your_task  ----------------------------> here your detected tasks will be shown                     '''

# workers routing

we can route our tasks to diffrent workers and routing could be based on load balance or task types

passing to queue:

1) we use -Q <queueme> flag to set a worker to watch a individuaqueue  celery -A <#filename> worker --loglevel=info -Qqueue2) in order to pass a task to a spicifiedqueuee this command in celery.py
    app.conf.task_routes = {'newapp.tasks.task1': {queue,}
    app.conf.task_routes = {'<#task location>': {queue'<queueme>'},}

3) if there is a independed worker we have to import task using this command
    app.conf.imports = ('newapp.tasks')
    app.conf.imports = ('<#task location>')
    +
    app.autodiscover_tasks()


